In [1]:
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Dynamically find the full path to the DB
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
db_path = os.path.join(project_root, "data", "games.db")

# Connect to the DB
conn = sqlite3.connect(db_path)

# Run the query
query = """
SELECT
    p.date,
    p.home_team,
    p.away_team,
    p.predicted_home_score,
    p.predicted_away_score,
    p.predicted_diff,
    p.win_probability,
    p.conf_low,
    p.conf_high,
    g.home_score AS actual_home_score,
    g.away_score AS actual_away_score,
    (g.home_score - g.away_score) AS actual_diff
FROM predictions p
JOIN games g
  ON p.date = g.date
 AND p.home_team = g.home_team
 AND p.away_team = g.away_team
WHERE g.home_score IS NOT NULL AND g.away_score IS NOT NULL
"""

df = pd.read_sql(query, conn)
conn.close()

# Compute metrics
df["error"] = (df["predicted_diff"] - df["actual_diff"]).abs()
df["winner_correct"] = (
    ((df["predicted_diff"] > 0) & (df["actual_diff"] > 0)) |
    ((df["predicted_diff"] < 0) & (df["actual_diff"] < 0))
)

# Show recent games
df.sort_values("date", ascending=False).head(10)


,date,home_team,away_team,predicted_home_score,predicted_away_score,predicted_diff,win_probability,conf_low,conf_high,actual_home_score,actual_away_score,actual_diff,error,winner_correct
0,2025-07-05,Indiana Fever,Los Angeles Sparks,79,83,-4.199485,1.0,-5.812103,-2.543804,87,89,-2,2.199485,True
1,2025-07-05,Minnesota Lynx,Golden State Valkyries,84,72,11.399674,1.0,9.773710,13.069045,82,71,11,0.399674,True
